# "2019 February One Day Holding Long Short Portfolio BackTest"
> "2019 February Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [3]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [4]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\DeLaMachine\\EDGAR\\1dayreturnpred2019train60K.csv')

In [5]:
#hide
df_testp=df_testp.loc[df_testp.Date>=20190201].loc[df_testp.Date<20190301].sort_values(by='Date',ascending=True).copy()

**Long-Short Portfolio**

In [6]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500
0,2019-02-01,1.0,0.113819,1.001138,1.006776,1.007001
1,2019-02-04,1.0,1.653080,1.017688,1.011517,1.013870
2,2019-02-05,-1.0,-0.443045,1.022197,1.009267,1.013023
3,2019-02-05,1.0,12.113263,1.146018,1.009267,1.013023
4,2019-02-06,-1.0,4.786690,1.091162,0.999823,1.004215
5,2019-02-06,1.0,-3.041522,1.057974,0.999823,1.004215
6,2019-02-07,-1.0,-0.600748,1.064329,1.000499,1.001693
7,2019-02-07,1.0,-7.840899,0.980876,1.000499,1.001693
8,2019-02-08,1.0,103.554708,1.996620,1.001208,0.999570
9,2019-02-11,-1.0,1.576912,1.965135,1.014114,1.014438


In [9]:
#hide_input
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3  

alt.LayerChart(...)

**Long portfolio**

In [10]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500
0,2019-02-01,1.0,0.113819,1.001138,1.006776,1.007001
1,2019-02-04,1.0,1.653080,1.017688,1.011517,1.013870
2,2019-02-05,1.0,12.113263,1.140963,1.009267,1.013023
3,2019-02-06,1.0,-3.041522,1.106260,0.999823,1.004215
4,2019-02-07,1.0,-7.840899,1.019520,1.000499,1.001693
5,2019-02-08,1.0,103.554708,2.075280,1.001208,0.999570
6,2019-02-11,1.0,-5.581939,1.959439,1.014114,1.014438
7,2019-02-12,1.0,-13.977391,1.685561,1.017181,1.019126
8,2019-02-13,1.0,-9.369567,1.527631,1.014483,1.014982
9,2019-02-14,1.0,25.064540,1.910525,1.025520,1.032691


In [13]:
#hide_input

a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3  

alt.LayerChart(...)

**Short Portfolio**

In [14]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500
0,2019-02-05,-1.0,-0.443045,1.004430,0.997776,0.999165
1,2019-02-06,-1.0,4.786690,0.956351,0.988439,0.990477
2,2019-02-07,-1.0,-0.600748,0.962097,0.989108,0.987990
3,2019-02-11,-1.0,1.576912,0.946925,1.002568,1.000560
4,2019-02-12,-1.0,-7.625178,1.019130,1.005600,1.005185
5,2019-02-13,-1.0,2.491958,0.993734,1.002933,1.001097
6,2019-02-14,-1.0,12.536583,0.869154,1.013844,1.018564
7,2019-02-15,-1.0,0.000000,0.869154,1.015363,1.018881
8,2019-02-19,-1.0,1.463905,0.856430,1.017168,1.021365
9,2019-02-20,-1.0,21.600723,0.671435,1.013581,1.017280


In [17]:
#hide_input

a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3  

alt.LayerChart(...)